In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

import wrangle
import my_toolkit


In [2]:
cluster1_cols = ['age','tax']
cluster2_cols = ['latitude','longitude','age', 'tax']

In [3]:
scale_cols = [
 'logerror',
 'bedroomcnt',
 'calc_bath',
 'structure_sqft',
 'fullbathcnt',
 'latitude',
 'longitude',
 'lot_sqft',
 'roomcnt',
 'tax_structure',
 'tax',
 'tax_land',
 'years_tax_delinquent',
 'bathroom_sum',
 'age',
 'logerror_upper',
 'logerror_lower',
 'tax_structure_ratio',
 'tax_land_ratio',
 ]

In [4]:
train, test, validate = wrangle.wrangle_zillow()

In [ ]:
def get_kmeans_clusters(df, cols, k, clusterer=None):
    if clusterer == None:
        from sklearn.cluster import KMeans
        clusterer = KMeans(n_clusters=k)
        clusterer.fit(df[cols])
    s = clusterer.predict(df[cols])
    return s